In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
mpl.rc('font', family='Malgun Gothic')

In [5]:
df_member = pd.read_csv('Member_Data.csv', encoding='EUC-KR')
df_product = pd.read_csv('Product_Data.csv')
df_sales = pd.read_csv('Sales_Data05.csv')

- 제품 고객 만족도, 재구매율

In [ ]:
# df_product의 '제품번호', '물품대분류' + df_sales의 '제품번호', '회원번호'로 고객의 제품 카테고리별 구매 횟수 추정?

- 고객 이탈률

In [27]:
# "회원상태" 하나 null
df_member['회원상태'].value_counts()

회원상태
정상회원     12380
탈퇴         156
탈퇴처리중        2
탈퇴신청         1
Name: count, dtype: int64

In [47]:
# '정상회원'이 아닐 경우 '고객 이탈'로 가정
cond_leave = df_member['회원상태'] != "정상회원"

leave_num = len(df_member.loc[cond_leave].index)
member_num = len(df_member['회원상태'])

customer_leave_rate = leave_num / member_num * 100
print(f"고객 이탈률은 {customer_leave_rate}% 이다.")

고객 이탈률은 1.2759170653907497% 이다.


- n개월 이상 구매 유지 고객비율

In [189]:
# 처음 주문과 가장 마지막 주문으로 n개월 이상 구매를 유지하는지 판단함.
# 주문 취소 여부는 고려하지 않음.
# 1개월 이상 구매 유지 고객 = 한 번 구매 후 아무 이력 없는 고객도 포함한다 가정. -> 즉, df_sales 데이터 내에서는 모든 고객이 1개월 이상 구매 유지 중.
# -> 회원가입만 하고 구매하지 않은 고객이 있을 가능성, df_member와 병합 후 계산할 필요가 있음.
df_sales['주문일시_dt'] = pd.to_datetime(df_sales['주문일시'])
df_sales['주문일시_year'] = df_sales['주문일시_dt'].dt.year
df_sales['주문일시_month'] = df_sales['주문일시_dt'].dt.month
df_sales['주문일시_day'] = df_sales['주문일시_dt'].dt.day

temp1 = df_sales.sort_values(by=['회원번호', '주문일시_month'])
df_orderdate = temp1.pivot_table(index = '회원번호', values = '주문일시_month', aggfunc = ['min', 'max']).reset_index()
# 연도는 모두 2021로 동일하므로 고려하지 않음.
df_orderdate['n개월구매유지'] = ( df_orderdate['max']['주문일시_month'] - df_orderdate['min']['주문일시_month'] )

n = int(input())
n -= 1
more_n_cond = df_orderdate['n개월구매유지'] >= n
keep_customer_num = len(df_orderdate.loc[more_n_cond].index)
customer_num = len(df_orderdate['회원번호'])
customer_keep_rate = keep_customer_num / customer_num * 100
print(f"{n+1}개월 이상 구매 유지 고객비율은 {customer_keep_rate}% 이다.")

 1


1개월 이상 구매 유지 고객비율은 100.0% 이다.


In [141]:
#df_sales['연'].value_counts()
#df_orderdate['n개월구매유지'].value_counts()

- 배송지연율

In [187]:
# 2일 이상 걸릴 소모 시 배송 지연으로 가정.
df_sales['배송시작일_dt'] = pd.to_datetime(df_sales['배송시작일'])
df_sales['배송시작일_year'] = df_sales['배송시작일_dt'].dt.year
df_sales['배송시작일_month'] = df_sales['배송시작일_dt'].dt.month
df_sales['배송시작일_day'] = df_sales['배송시작일_dt'].dt.day

df_sales['배송완료일_dt'] = pd.to_datetime(df_sales['배송완료일'])
df_sales['배송완료일_year'] = df_sales['배송완료일_dt'].dt.year
df_sales['배송완료일_month'] = df_sales['배송완료일_dt'].dt.month
df_sales['배송완료일_day'] = df_sales['배송완료일_dt'].dt.day

# 배송 연도는 모두 2021로 동일하므로 고려하지 않음, 윤년을 고려하여 코드를 수정할 필요가 있음.
# 배송 완료 후 월이 넘어갈 때,, 오류O
#date_list = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
#if (df_sales['배송완료일_month'] > df_sales['배송시작일_month']) :
#    df_sales['배송소모일'] = ( date_list[df_sales['배송시작일_month']] - df_sales['배송시작일_day'] ) + df_sales['배송완료일_day'] + ( (df_sales['배송완료일_month'] - df_sales['배송시작일_month'] - 1) * 30 )
df_sales['배송소모일'] = df_sales['배송완료일_day'] - df_sales['배송시작일_day']
delivery_late_cond = df_sales['배송소모일'] >= 2
delivery_late_num = len(df_sales.loc[delivery_late_cond].index)

# 전체 배송 건 수
delivery_cond = pd.notna(df_sales['배송시작일'])
total_delivery_num = len(df_sales.loc[delivery_cond].index)

delivery_late_rate = delivery_late_num / total_delivery_num * 100
print(f"배송지연율은 {delivery_late_rate}% 이다.")

배송지연율은 14.20210329486914% 이다.


In [185]:
#df_sales['배송시작일_year'].value_counts()
#df_sales['배송완료일_year'].value_counts()
#df_sales.info()